<a href="https://colab.research.google.com/github/abhinavdayal/EVA4/blob/master/S4/EVA4_Session_2_Success01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# call the heavenly gods
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, 3, padding=1, bias=False) #input 28x28x1, output 28x28x10, RF 3
        self.bn1 = nn.BatchNorm2d(10)
        self.d1 = nn.Dropout(0.05) #5 % probability 

        self.conv2 = nn.Conv2d(10, 10, 3, padding=1, bias=False) #input 28x28x10, output 28x28x10, RF 5
        self.bn2 = nn.BatchNorm2d(10)
        self.d2 = nn.Dropout(0.05) #5 % probability 

        self.pool1 = nn.MaxPool2d(2, 2) # RF 6

        self.conv3 = nn.Conv2d(10, 16, 3, padding=1, bias=False) #input 14x14x10, output 14x14x16, RF 10
        self.bn3 = nn.BatchNorm2d(16)
        self.d3 = nn.Dropout(0.05) #5 % probability 

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1, bias=False) #input 14x14x16, output 14x14x16, RF 14
        self.bn4 = nn.BatchNorm2d(16)
        self.d4 = nn.Dropout(0.05) #5 % probability 

        self.pool2 = nn.MaxPool2d(2, 2) # RF 16
        
        self.conv5 = nn.Conv2d(16, 32, 3, padding=1, bias=False) #input 7x7x16, output 7x7x32, RF 24
        self.bn5 = nn.BatchNorm2d(32)
        self.d5 = nn.Dropout(0.05) #5 % probability 

        self.conv6 = nn.Conv2d(32, 32, 3, bias=False) #input 7x7x32, output 5x5x32, RF 32
        self.bn6 = nn.BatchNorm2d(32)
        self.d6 = nn.Dropout(0.05) #5 % probability 
        
        self.conv7 = nn.Conv2d(32, 10, 1, bias=False) #input 5x5x32, output 5x5x10, RF 32

    def forward(self, x):
        x = self.pool1(F.relu(self.bn2(self.d2(self.conv2(F.relu(self.bn1(self.d1(self.conv1(x)))))))))
        x = self.pool2(F.relu(self.bn4(self.d4(self.conv4(F.relu(self.bn3(self.d3(self.conv3(x)))))))))
        x = self.conv7(F.relu(self.bn6(self.d6(self.conv6(F.relu(self.bn5(self.d5(self.conv5(x)))))))))
        x = F.avg_pool2d(x, 5)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
           Dropout-2           [-1, 10, 28, 28]               0
       BatchNorm2d-3           [-1, 10, 28, 28]              20
            Conv2d-4           [-1, 10, 28, 28]             910
           Dropout-5           [-1, 10, 28, 28]               0
       BatchNorm2d-6           [-1, 10, 28, 28]              20
         MaxPool2d-7           [-1, 10, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           1,456
           Dropout-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
          Dropout-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [10]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm
from tqdm import tnrange, tqdm_notebook
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    train_loss = 0
    correct = 0
    pbar =  tqdm_notebook(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        train_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()

        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')

    train_loss /= len(train_loader.dataset)

    print('Train set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        train_loss, correct, len(train_loader.dataset),
        100. * correct / len(train_loader.dataset)))


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [12]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in tnrange(1, 21):
    print("\nEpoch:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)


Epoch: 1


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:43: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train set: Average loss: 0.5383, Accuracy: 52188/60000 (86.98%)
Test set: Average loss: 0.0974, Accuracy: 9788/10000 (97.88%)

Epoch: 2


Train set: Average loss: 0.0831, Accuracy: 58671/60000 (97.78%)
Test set: Average loss: 0.0471, Accuracy: 9873/10000 (98.73%)

Epoch: 3


Train set: Average loss: 0.0553, Accuracy: 59075/60000 (98.46%)
Test set: Average loss: 0.0432, Accuracy: 9887/10000 (98.87%)

Epoch: 4


...

Train set: Average loss: 0.0188, Accuracy: 59659/60000 (99.43%)
Test set: Average loss: 0.0187, Accuracy: 9937/10000 (99.37%)

Epoch: 17


Train set: Average loss: 0.0179, Accuracy: 59672/60000 (99.45%)
Test set: Average loss: 0.0177, Accuracy: 9941/10000 (99.41%)

Epoch: 18


Train set: Average loss: 0.0174, Accuracy: 59673/60000 (99.45%)
Test set: Average loss: 0.0212, Accuracy: 9920/10000 (99.20%)

Epoch: 19


Train set: Average loss: 0.0154, Accuracy: 59717/60000 (99.53%)
Test set: Average loss: 0.0163, Accuracy: 9945/10000 (99.45%)

Epoch: 20


Train set: Average loss: 0.0155, Accuracy: 59722/60000 (99.54%)
Test set: Average loss: 0.0159, Accuracy: 9948/10000 (99.48%)

